In [ ]:
# BaseballCV Command Analyzer Example

# @title 1. Install BaseballCV

print("Installing dependencies...")
!pip install git+https://github.com/Jensen-holm/statcast-era-pitches.git@1.1 -q
# No MediaPipe needed

!git clone https://github.com/dylandru/BaseballCV.git
%cd BaseballCV
!git checkout 122-create-command-estimator-for-baseballtools-class # <-- Branch with latest CommandAnalyzer
!pip install -e . -q
!pip install -q git+https://github.com/dylandru/yolov9.git

print("\n--- Installation Complete ---")
print("IMPORTANT: If dependency errors occur, RESTART RUNTIME and run again.")
print("-------------------------------\n")
print("BaseballCV installation complete (or already installed).")

In [ ]:
# @title 2. Import BaseballTools and Set Up Directories

import os
from baseballcv.functions import BaseballTools

# Define directories for our outputs
# GloveTracker will create a 'glove_tracking_results' subdirectory within this.
BASE_OUTPUT_DIR = "baseball_analysis_output"
GLOVE_TRACKER_CSVS_DIR = os.path.join(BASE_OUTPUT_DIR, "glove_tracking_results") # Default output of GloveTracker
COMMAND_ANALYSIS_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "command_analysis_results") # For CommandAnalyzer outputs
VIDEO_OVERLAYS_DIR = os.path.join(COMMAND_ANALYSIS_OUTPUT_DIR, "overlays") # For CommandAnalyzer video overlays

# Create directories if they don't exist
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
os.makedirs(COMMAND_ANALYSIS_OUTPUT_DIR, exist_ok=True) # CommandAnalyzer will use csv_input_dir for its main CSVs
os.makedirs(VIDEO_OVERLAYS_DIR, exist_ok=True) # For video overlays specifically

print(f"Base output directory: {BASE_OUTPUT_DIR}")
print(f"GloveTracker CSVs will be in: {GLOVE_TRACKER_CSVS_DIR}") # This is where track_gloves will save, and analyze_pitcher_command will read from
print(f"Command analysis CSVs will be in: {COMMAND_ANALYSIS_OUTPUT_DIR}")
print(f"Command analysis video overlays will be in: {VIDEO_OVERLAYS_DIR}")

In [ ]:
# @title 3. Generate Glove Tracking Data (Prerequisite for CommandAnalyzer)
# CommandAnalyzer relies on CSV files generated by the GloveTracker.
# We'll use the 'scrape' mode of `track_gloves` to download a couple of videos
# from Baseball Savant and process them. This will create the necessary CSVs.

# Initialize BaseballTools - use 'cpu' for Colab unless a GPU is available and configured.
# If you have a GPU runtime, you can try 'cuda'.
baseball_tools = BaseballTools(device='cpu', verbose=True)

print("Running GloveTracker to generate input CSVs for CommandAnalyzer...")
print("This step will download videos and process them. It might take a few minutes.")

# --- Parameters for Glove Tracking ---
# We'll scrape 1 video for a quick demonstration.
# In a real scenario, you'd process many videos.
glove_tracking_results = baseball_tools.track_gloves(
    mode="scrape",
    start_date="2024-04-17", # Use a recent date for likely available videos
    end_date="2024-04-17",
    max_videos=3,            # Limit to 1 video for this demo
    max_videos_per_game=1,
    create_video=True,       # Creates the tracked video (optional, but good for verification)
    generate_heatmap=True,   # Generates a heatmap (optional)
    # GloveTracker will save its outputs (CSVs, videos, heatmaps)
    # into a 'glove_tracking_results' folder by default relative to where it's run
    # or based on its results_dir. We will point CommandAnalyzer to this.
    # For this example, let's ensure it uses our BASE_OUTPUT_DIR context.
    # The GloveTracker is initialized within BaseballTools, and its default `results_dir`
    # is "glove_tracking_results". If BaseballTools is run from a notebook,
    # this folder will appear in the root of the Colab environment.
    # We have set GLOVE_TRACKER_CSVS_DIR to reflect this default.
)

if "error" in glove_tracking_results:
    print(f"\nError during glove tracking: {glove_tracking_results['error']}")
else:
    print("\nGlove tracking finished.")
    if glove_tracking_results.get("processed_videos", 0) > 0:
        print(f"Successfully processed {glove_tracking_results['processed_videos']} video(s).")
        print(f"CSVs and other outputs saved in: {glove_tracking_results.get('results_dir', 'glove_tracking_results (default)')}")
        # CommandAnalyzer will need the path to the directory containing the CSVs.
        # track_gloves in 'scrape' or 'batch' mode returns 'results_dir' which is where GloveTracker saves everything.
        ACTUAL_GLOVE_TRACKER_OUTPUT_DIR = glove_tracking_results.get('results_dir', GLOVE_TRACKER_CSVS_DIR)
        print(f"CommandAnalyzer will look for CSVs in: {ACTUAL_GLOVE_TRACKER_OUTPUT_DIR}")
    else:
        print("No videos were processed by GloveTracker. CommandAnalyzer may not have input.")
        ACTUAL_GLOVE_TRACKER_OUTPUT_DIR = GLOVE_TRACKER_CSVS_DIR # Fallback

In [ ]:
# @title 4. Analyze Pitcher Command
# Now that we have glove tracking CSVs, we can use CommandAnalyzer.
should_create_overlays = True
if "error" not in glove_tracking_results and glove_tracking_results.get("processed_videos", 0) > 0:
    print(f"\nRunning CommandAnalyzer on CSVs from: {ACTUAL_GLOVE_TRACKER_OUTPUT_DIR}")



    # --- Parameters for Command Analysis ---
    command_analysis_df = baseball_tools.analyze_pitcher_command(
        csv_input_dir=ACTUAL_GLOVE_TRACKER_OUTPUT_DIR, # Directory containing GloveTracker CSVs
        output_csv="pitcher_command_summary.csv", # Main output CSV filename (will be saved in csv_input_dir)
        create_overlay=should_create_overlays,                      # Create videos with target/actual overlays
        video_output_dir=VIDEO_OVERLAYS_DIR,      # Where to save the overlay videos
        debug_mode=False,                         # Set to True for more detailed logs and plots from CommandAnalyzer
        group_by_agg=['pitcher', 'pitch_type'],   # Columns for aggregate metrics
        cmd_threshold_inches=6.0                  # Threshold for "commanded" pitch
    )

    if not command_analysis_df.empty:
        print("\nCommand analysis complete!")
        print("Detailed results DataFrame (first 5 rows):")
        print(command_analysis_df.head())

        # CommandAnalyzer saves its main detailed CSV inside the `csv_input_dir`
        # and the aggregate CSV also inside `csv_input_dir` prefixed with `aggregate_`.
        print(f"\nDetailed command CSV saved as 'pitcher_command_summary.csv' inside '{ACTUAL_GLOVE_TRACKER_OUTPUT_DIR}'")
        print(f"Aggregate command CSV saved as 'aggregate_pitcher_command_summary.csv' inside '{ACTUAL_GLOVE_TRACKER_OUTPUT_DIR}'")
        if should_create_overlays:
            print(f"Overlay videos saved in: {VIDEO_OVERLAYS_DIR}")
    else:
        print("\nCommand analysis did not produce results. This might be due to issues in the input CSVs or lack of data.")
else:
    print("\nSkipping CommandAnalyzer because GloveTracker did not process any videos or encountered an error.")